# Time-series signatures from raw brdf imagery / smoothed ts outputs

In [ ]:
import os
import sys
from pathlib import Path
import rasterio as rio
from rasterio import plot
import shutil
import tempfile
import json
import random
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from pyproj import Proj, transform
from pyproj import CRS
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry import Polygon
from rasterio.plot import show
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
%matplotlib notebook

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from ts_profile import *
from ts_composite import *

In [ ]:
##### Note: 
###  Notebook is housed on login node; time series calculations shold not be run on login node (and take a long time anyway). 
###  Better to install on cluster, run GettimeSeries.sh in Bash folder and copy output file path to parameters 
###  to read with `LoadTSfromFile(TSfile)` to load time series here for plotting.

###TODO: Enable editing Bash script to run through Slurm via this notebook directly

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basic_config
print("Run Parameters: \n raw_dir = {} \n  index_dir = {} \n local_dir = {} \n grid_cell = {} \n years = {} \n spec_index = {}"
      .format(basic_config['raw_dir'],basic_config['index_dir'], basic_config['local_dir'], basic_config['grid_cell'],
             basic_config['yr_range'], basic_config['spec_index']))
%store -r plot_params
%store -r timeseries_params
print('filter class: {}'.format(timeseries_params['filter_class']))
if timeseries_params['load_prerunTS'] == True:
    print(" Smooth TS loaded from {}".format(timeseries_params['smooth_TSfile']))
    print(" Raw TS loaded from {}".format(timeseries_params['raw_TSfile']))
elif timeseries_params['load_samp']==True:
    print("using point_file: {}".format(basic_config['ptfile']))
else:
    print("using poly_file: {}".format(basic_config['polyfile']))
    

In [ ]:
if plot_params['interactive'] == True:
    from ipywidgets import Label
    from ipyleaflet  import Map, GeoData, basemaps, LayersControl, ImageOverlay, Marker, Popup
    from localtileserver import get_leaflet_tile_layer, TileClient
    from plot_interactive import *

In [ ]:
#natpts = get_pts_in_grid(basic_config['grid_file'], basic_config['grid_cell'], basic_config['ptfile'])
#natpts.head(n=5)

## Plot smooth time series

In [ ]:
#smooth = load_ts_from_file('../samp_data/ts_3737_rice')

# If time series is already saved to file, load it here
if timeseries_params['load_prerunTS'] == True:
    if timeseries_params['smooth_TSfile']:
        smooth = load_ts_from_file(timeseries_params['smooth_TSfile'])
        
## Else generate time series for selected coordinates,sample points (if load_samp parameter == True), 
## or random points in sample polygons

else:
    if str(basic_config['ptfile']).endswith('SelectedCoords.csv'):
        gcells = [basic_config['grid_cell']]
        load_samp = True
        filter_class = None
    else:
        gcells = basic_config['grid_cells']
        load_samp=timeseries_params['load_samp']
        ## for point sample, can filter on Class column to restrict sample to a certain land cover type
        filter_class = timeseries_params['filter_class'] 
        
    smooth = get_timeseries_for_pts_multicell(basic_config['local_dir'], basic_config['spec_index'], 
                                              timeseries_params['start_yr'], timeseries_params['end_yr'],
                                              basic_config['smooth_dir'], 'Smooth', basic_config['grid_file'], gcells, 
                                              polyfile=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'],
                                              newest=timeseries_params['newest_samp'], npts=timeseries_params['npts'], 
                                              seed=timeseries_params['seed1'], load_samp = load_samp, ptfile=basic_config['ptfile'], 
                                              filter_class = filter_class)

In [ ]:
pd.DataFrame.to_csv(smooth, '../samp_data/ts_3737_rice', sep=',', na_rep='NaN')
print(smooth)

In [ ]:
smooth = smooth[smooth['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
#plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Smoothed {} for sample {}-{}'.format(basic_config['spec_index'],timeseries_params['start_yr'],timeseries_params['end_yr']))
plt.ylim(0,7000)
plt.show();

In [ ]:
smooth = smooth[smooth['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
for r in range(smooth.shape[0]):
    line, = plt.plot(smooth.index, smooth[r], color=(random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)), label=r,  picker=True, pickradius=5)

text = plt.text(0,0,"")
check_pts = []
def onpick(event):
    thisline = event.artist
    ind = event.ind
    xdata = thisline.get_xdata()
    ydata = thisline.get_ydata()
    text.set_position((xdata[ind], ydata[ind]))
    text.set_text(event.artist.get_label())
    check_pts.append(event.artist.get_label())

fig.canvas.mpl_connect('pick_event', onpick)

plt.plot(smooth.index, smooth['ALL'], color='black', linewidth=7.0)
plt.title('Smoothed {} for {} sample points {}-{}'.format(basic_config['spec_index'],timeseries_params['filter_class'],timeseries_params['start_yr'],timeseries_params['end_yr']))
plt.ylim(0,7000)
plt.show()

In [ ]:
print(check_pts)

In [ ]:
si_vars = ['maxv_yr','minv_yr','amp_yr']
#si_vars = ['Nov_20','Dec_20','Feb_20']

comp_dir = os.path.join(basic_config['smooth_dir'],'{:06d}'.format(basic_config['grid_cell']),'comp')
comp_path = os.path.join(comp_dir,'{:06d}_{}_{}_{}{}{}.tif'.format(basic_config['grid_cell'],basic_config['filter_yr'],
                                                                  basic_config['spec_index'],si_vars[0],si_vars[1],si_vars[2]))
if Path(comp_path).exists():
    print('comp already exists')
    comp = comp_path
else:
    comp = make_ts_composite(basic_config['grid_cell'],basic_config['index_dir'],comp_dir,basic_config['filter_yr'],
                             timeseries_params['start_mo'],basic_config['spec_index'],si_vars)

In [ ]:
point_df = pd.read_csv(basic_config['ptfile'], index_col=0)
selpts = point_df[point_df['Class']==timeseries_params['filter_class']]
ptsdf = get_pts_in_grid (basic_config['grid_file'], basic_config['grid_cell'], selpts)
ptsdf.reset_index(drop=False, inplace=True)
ptsdf_ll = ptsdf.to_crs("EPSG:4326")

In [ ]:
tile_client = TileClient(comp,port=5554)
m = Map(center=tile_client.center(), zoom=12, basemap=basemaps.Esri.WorldImagery)
t = get_leaflet_tile_layer(tile_client, band=[3,2,1])
#m.add_layer(t)

pts_data = GeoData(geo_dataframe = ptsdf_ll, style={'color': 'black', 'radius':3, 'fillColor': 'yellow'},
    point_style={'radius': 5}, name = 'OID_')
m.add(pts_data)

for i in check_pts:
    # Get the coordinates and place name from the feature properties
    coords = ptsdf_ll['geometry'].iloc[int(i)]
    place_name = ptsdf_ll['OID_'].iloc[int(i)]
    marker = Marker(location = [coords.y,coords.x])
    popup = Popup(
        location=[coords.y,coords.x],
        close_button=False,
        auto_close=False,
        close_on_click=False,
        close_on_escape_key=False,
        )
    popup.children = "{}:OID={} coords={},{}".format(i,place_name,coords.y,coords.x)
    print(popup.children)
    message=Label()
    message.value = popup.children
    marker.popup = message
    m.add_layer(marker)
    
m.add(LayersControl())
display(m)

## Examine phenology variables for a time series

In [ ]:
comp_dir = os.path.join(basic_config['smooth_dir'],'{:06d}'.format(basic_config['grid_cell']),'comp/{}'.format( basic_config['spec_index']))
temp = 'wet'
phen_bands = [f'maxv_{temp}',f'maxd_{temp}',f'sosd_{temp}',f'sosv_{temp}',f'eosd_{temp}',f'eosv_{temp}',f'rog_{temp}',f'ros_{temp}',f'los_{temp}']
comp_path = os.path.join(comp_dir,'{:06d}_{}_{}_Phen{}.tif'.format(basic_config['grid_cell'],basic_config['filter_yr'],
                                                                   basic_config['spec_index'],temp))
if Path(comp_path).exists():
    print('comp already exists')
    phen_comp = comp_path
else:
    phen_comp = make_ts_composite(basic_config['grid_cell'], basic_config['index_dir'],
                                      comp_dir,basic_config['filter_yr'],timeseries_params['start_mo'],
                                      basic_config['spec_index'],phen_bands)

In [ ]:
#[maxv_wet', 'maxd_wet', 'sosv_wet', 'sosd_wet', 'rog_wet', 'eosv_wet', 'eosd_wet', 'ros_wet', 'los_wet']
vals = get_values_at_coords(ptsdf, plot_params['inputCRS'], phen_comp,
                           ['maxv_wet', 'maxd_wet', 'sosv_wet', 'sosd_wet', 'rog_wet', 'eosv_wet', 'eosd_wet', 'ros_wet', 'los_wet'])
print(vals)        

In [ ]:
print(vals.at[12,'sosd_wet'])

In [ ]:
samppt = 7
if vals.at[samppt,'sosd_wet'] < 366:
    sosd = datetime.datetime(basic_config['filter_yr'], 1, 1) + datetime.timedelta(int(vals.at[samppt,'sosd_wet']) - 1)
else:
    sosd = datetime.datetime(basic_config['filter_yr'] + 1, 1, 1) + datetime.timedelta(int(vals.at[samppt,'sosd_wet'] -365))  
sosv = int(vals.at[samppt,'sosv_wet'])

if vals.at[samppt,'eosd_wet'] < 366:
    eosd = datetime.datetime(basic_config['filter_yr'], 1, 1) + datetime.timedelta(int(vals.at[samppt,'eosd_wet']) - 1)
else:
    eosd = datetime.datetime(basic_config['filter_yr'] + 1, 1, 1) + datetime.timedelta(int(vals.at[samppt,'eosd_wet'] -365))
eosv = int(vals.at[samppt,'eosv_wet'])

if vals.at[samppt,'maxd_wet'] < 366:
    posd = datetime.datetime(basic_config['filter_yr'], 1, 1) + datetime.timedelta(int(vals.at[samppt,'maxd_wet']) - 1)
else:
    posd = datetime.datetime(basic_config['filter_yr'] + 1, 1, 1) + datetime.timedelta(int(vals.at[samppt,'maxd_wet'] - 365))
posv = int(vals.at[samppt,'maxv_wet'])

fig = plt.figure(figsize=(10, 5))
plt.plot(smooth.index, smooth[samppt], color='black', linewidth=1.0)
plt.plot(sosd,sosv,'ro')
plt.annotate('SOS', xy=(sosd,sosv-300))
plt.plot(eosd,eosv,'ro')
plt.annotate('EOS', xy=(eosd,eosv-400))
plt.plot(posd,posv,'ro')
plt.annotate('POS', xy=(posd,posv+200))
x = [sosd, eosd]
y = [sosv, sosv]
plt.plot(x, y, 'r')
avgd = (eosd - sosd) / 2
plt.annotate('LOS',xy=(sosd + avgd, sosv + 300))
plt.title('Example of smoothed {} curve with phenology points'.format(basic_config['spec_index']))
plt.ylim(0,7000)
plt.show()

## To compare two smoothed time-series (i.e. with different parameters)
One is in the regular time_series folder called 'brdf_ts/ms/index' and the other is in folder called 'brdf_ts_old/ms/index'  

In [ ]:
if not os.path.exists(os.path.join(basic_config['smooth_dir'],'{:06d}'.format(basic_config['grid_cell']),'brdf_ts_old','ms',basic_config['spec_index'])):
    print('There is no old time series to compare')
else:
    if basic_config['ptfile'].endswith('SelectedCoords.csv'):
        gcells = [basic_config['grid_cell']]
        load_samp = True
    else:
        gcells = basic_config['grid_cells']
        load_samp=timeseries_params['load_samp']
        
    old = get_timeseries_for_pts_multicell(basic_config['local_dir'], basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['smooth_dir'], 'Smooth_old', basic_config['grid_file'], gcells, 
            ground_polys=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp, basic_config['ptfile'])

    old = old[old['ALL'] != 0]

    fig = plt.figure(figsize=(10, 5))
    plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
    plt.errorbar(old.index, old['ALL'], yerr=old['stdv'], fmt='o', color='g')
    plt.title('New vs old smoothed ts {} for sample {}-{}'.format(basic_config['spec_index'],timeseries_params['start_yr'],timeseries_params['end_yr']))
    plt.ylim(0,7000)
    plt.show();

## Plot raw time series

In [ ]:
if timeseries_params['load_prerunTS'] == True:
    if timeseries_params['raw_TSfile']:
        raw = load_TSfromFile(timeseries_params['raw_TSfile'])
else:
    raw = get_timeseries_for_pts_multicell(basic_config['home_dir'], basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['raw_dir'], timeseries_params['image_type'], basic_config['grid_file'], timeseries_params['grid_cells'], 
            ground_polys=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp=timeseries_params['load_samp'], ptfile=basic_config['ptfile'])

In [ ]:
raw = raw[raw['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
#plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Raw {} for sample points {}-{}'.format(basic_config['spec_index'], timeseries_params['start_yr'],timeseries_params['end_yr']))
plt.show()

## To plot a timeseries for each polygon in file:

In [ ]:
smooth = get_timeseries_for_pts_multicell(basic_config['home_dir'], basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['smooth_dir'], 'Smooth', basic_config['grid_file'], timeseries_params['grid_cells'], 
            ground_polys=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp=timeseries_params['load_samp'], basic_config['ptfile'])

In [ ]:
smooth

In [ ]:
smooth = smooth[smooth['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
#plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
plt.plot(smooth.index, smooth[22], color='k')
#plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Smoothed {} for natural forest points {}-{}'.format(basic_config['spec_index'],timeseries_params['start_yr'],timeseries_params['end_yr']))
plt.show();

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

out_name = str(basic_config['country']+'2_TimeSeries'+str(basic_config['grid_cell'])+'_from_'+str(timeseries_params['start_yr'])+'_to_'+str(timeseries_params['end_yr']))
!jupyter nbconvert --output-dir='../outputs' --to html --no-input --output=$out_name 2_TimeSeriesSignatures.ipynb